# Calculating maximum yield of flaviolin in P. putida 

Here we will use a genome-scale model to predict the maximum yield of flaviolin in P. putida w/ the alternative malonyl-CoA pathway

This notebook has NOT been tested on [jprime.lbl.gov](jprime.lbl.gov) with the **biodesign_3.7** kernel.

# Setup

In [1]:
import cobra
from cobra.flux_analysis import flux_variability_analysis

import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import Image                                                 # Used to display images

# Getting and preparing the genome-scale model

## Load model

In [2]:
file_name = 'iJN1463_modifiedv2.json'
model = cobra.io.load_json_model(file_name)   # Model from BIGG slightly improved by Joonhoon
cobra.io.write_sbml_model(model, "iJN1463_modified.xml")

## Add pathway to model

The model already has all the reactions up to malonyl-CoA:

### Exogenous pathway

**1-)** malonyl-CoA to 1,3,6,8-THN:
    create THN metabolite

In [3]:
thn_c = cobra.Metabolite(
    'thn_c',
    formula='C10H8O4',
    name='1,3,6,8-Naphthalenetetrol',
    compartment='c')
thn_c.charge = 0
THNS = cobra.Reaction('THNS')
THNS.name = 'tetrahydroxynaphthalene synthase'
THNS.subsystem = 'flaviolin biosynthesis'
THNS.lower_bound = 0.  # This is the default
THNS.upper_bound = 1000.  # This is the default
mal_c = model.metabolites.get_by_id('malcoa_c')
co2_c = model.metabolites.get_by_id('co2_c')
coa_c = model.metabolites.get_by_id('coa_c')
h_c = model.metabolites.get_by_id('h_c')
h2o_c = model.metabolites.get_by_id('h2o_c')

THNS.add_metabolites({
    mal_c: -5.0,
    h_c: -5,
    thn_c: 1.0,
    co2_c: 5.0,
    coa_c: 5.0,
    h2o_c: 1
})
h_c.charge=1
THNS.gene_reaction_rule = 'rppA'
THNS.check_mass_balance()

{}

In [4]:
model.add_reactions([THNS])

In [5]:
model.reactions.THNS

Reaction identifier,THNS
Name,tetrahydroxynaphthalene synthase
Memory address,0x07fdde56df430
Stoichiometry,"5 h_c + 5.0 malcoa_c --> 5.0 co2_c + 5.0 coa_c + h2o_c + thn_c 5 H+ + 5.0 Malonyl CoA C24H33N7O19P3S --> 5.0 CO2 CO2 + 5.0 Coenzyme A + H2O H2O + 1,3,6,8-Naphthalenetetrol"
GPR,rppA
Lower bound,0.0
Upper bound,1000.0


adding oxaloacetate and acetyl-Coa to malonyl-CoA and pyruvate

In [6]:
MCCT = cobra.Reaction('MCCT')
MCCT.name = 'malonyl-CoA carboxytransferase'
MCCT.subsystem = 'alt-mal-coa'
MCCT.lower_bound = 0.  # This is the default
MCCT.upper_bound = 1000.  # This is the default
ace_c = model.metabolites.get_by_id('accoa_c')
oaa_c = model.metabolites.get_by_id('oaa_c')
pyr_c = model.metabolites.get_by_id('pyr_c')

MCCT.add_metabolites({
    ace_c: -1.0,
    oaa_c: -1,
    mal_c: 1.0,
    pyr_c: 1.0
})

MCCT.gene_reaction_rule = 'mmda'
MCCT.check_mass_balance()
model.add_reactions([MCCT])
model.reactions.MCCT

Reaction identifier,MCCT
Name,malonyl-CoA carboxytransferase
Memory address,0x07fdde5688dc0
Stoichiometry,accoa_c + oaa_c --> malcoa_c + pyr_c Acetyl-CoA + Oxaloacetate --> Malonyl CoA C24H33N7O19P3S + Pyruvate
GPR,mmda
Lower bound,0.0
Upper bound,1000.0


In [21]:
PCKR = cobra.Reaction('PCKR')
PCKR.name = 'pep carboxykinase reverse'
PCKR.subsystem = 'alt-mal-coa'
PCKR.lower_bound = 0.  # This is the default
PCKR.upper_bound = 1000.  # This is the default
pep_c = model.metabolites.get_by_id('pep_c')
adp_c = model.metabolites.get_by_id('adp_c')
atp_c = model.metabolites.get_by_id('atp_c')

PCKR.add_metabolites({
    pep_c: -1.0,
    adp_c: -1,
    co2_c: -1.0,
    oaa_c: 1.0,
    atp_c: 1.0
})

PCKR.gene_reaction_rule = 'ASPEPCK'
PCKR.check_mass_balance()
model.add_reactions([PCKR])
model.reactions.PCKR

{}

**2-)** adding THN oxidation to flaviolin:

Creating flaviolin metabolite first:

In [8]:
flavio_c = cobra.Metabolite(
    'flavio_c',
    formula='C10H6O5',
    name='Flaviolin',
    compartment='c')
flavio_c.charge = 0

And now creating the reaction and adding it to the model:

In [9]:
FLAS= cobra.Reaction('FLAS')
FLAS.name = 'tetrahydroxynaphthalene oxidation'
FLAS.subsystem = 'flaviolin biosynthesis'
FLAS.lower_bound = 0.  # This is the default
FLAS.upper_bound = 1000.  # This is the default

In [10]:
o2_c = model.metabolites.o2_c
FLAS.add_metabolites({
    o2_c: -1.0,
    thn_c: -1.0,
    flavio_c: 1.0,
    h2o_c: 1.0
})

In [11]:
FLAS.check_mass_balance()

{}

In [12]:
model.add_reactions([FLAS])
model.reactions.FLAS

Reaction identifier,FLAS
Name,tetrahydroxynaphthalene oxidation
Memory address,0x07fdde56acaf0
Stoichiometry,"o2_c + thn_c --> flavio_c + h2o_c O2 O2 + 1,3,6,8-Naphthalenetetrol --> Flaviolin + H2O H2O"
GPR,
Lower bound,0.0
Upper bound,1000.0


In [13]:
model.add_boundary(flavio_c, type='demand')

Reaction identifier,DM_flavio_c
Name,Flaviolin demand
Memory address,0x07fdde570b670
Stoichiometry,flavio_c --> Flaviolin -->
GPR,
Lower bound,0
Upper bound,1000.0


In [14]:
cobra.io.save_json_model( model, "iJN1463_mod_flavio.json")
cobra.io.write_sbml_model(model, "iJN1463_mod_flavio.xml")

## Find maximum yield

Set model objetive to the exchange reaction for the final product (so as to maximize its production):

In [15]:
model.objective = model.reactions.DM_flavio_c

In [16]:
model.reactions.DM_flavio_c

Reaction identifier,DM_flavio_c
Name,Flaviolin demand
Memory address,0x07fdde570b670
Stoichiometry,flavio_c --> Flaviolin -->
GPR,
Lower bound,0
Upper bound,1000.0


Let's optimize product formation:

In [17]:
solution = model.optimize()

The maximum production in mMol/gdw/hr for 4-ACA is:

In [18]:
solution.fluxes['DM_flavio_c']

2.8180571428571395

This number increases with the glucose input:

In [19]:
solution.fluxes['EX_glc__D_e']

-6.0

but the ratio (i.e. the maximum theoretical yield) is constant:

In [20]:
abs(solution.fluxes['DM_flavio_c']/solution.fluxes['EX_glc__D_e'])

0.4696761904761899

In [32]:
solution = model.optimize()
print('complete model: ', abs(solution.fluxes['DM_flavio_c']/solution.fluxes['EX_glc__D_e']))
with model:
    model.reactions.ACCOAC.knock_out()
    solution = model.optimize()
    print('without native malonyl-CoA production: ', abs(solution.fluxes['DM_flavio_c']/solution.fluxes['EX_glc__D_e']))
    model.reactions.PCKR.knock_out()
    solution = model.optimize()
    print('without the new pathway to malonyl-CoA: ', abs(solution.fluxes['DM_flavio_c']/solution.fluxes['EX_glc__D_e']))

complete model:  0.46967619047619297
without native malonyl-CoA production:  0.46967619047619297
without the new pathway to malonyl-CoA:  0.4550990990990993


These results mean that the alternative pathway to malonyl-CoA could replace the endogenous (native) pathway

in mMol of 4-ACA/ mMol of glucose. 

For our 20mM glucose, this corresponds to 9mM of flaviolin, i.e. 1.8 g/L